In [28]:
import os #add sys.arg dans le standard import
import numpy as np
#Vérifier ce que les 2 librairies suivantes comprennent et ajuster en conséquence
import plotly.express as px
import plotly.graph_objects as go
from scipy import integrate
#todo : tout rephraser plus explicatif en texte continu et faire un meilleur cell template de standard import

La conversion d'une grandeur radiométrique $X_e$ en grandeur photométrique $X_v$ sur un intervalle infitésimal $\mathrm{d}\lambda$ se fait avec la différentielle
\begin{equation}
\mathrm{d}X_v=K_{max}X_e(\lambda)V(\lambda)\,\mathrm{d}\lambda=683\,X_e(\lambda)V(\lambda)\,\mathrm{d}\lambda
\end{equation}
avec un maximum d'efficacité de conversion $K_{max}=683$ lumen/W de l'oeil humain en vision adaptée de jour (photopique). Ce maximum situé à 555 nm se trouve normalisé à 1 dans le coefficient d'efficacité lumineuse spectrale $V(\lambda)$ photopique relatif, tiré de https://commons.wikimedia.org/wiki/File:Luminosity.png avec un coup de pouce de https://automeris.io/wpd/ :

In [29]:
#todo: remplacer par les données et/ou fonctions de http://www.cvrl.org/people/Stockman/pubs/2005%20Vstar%20SSJJ.pdf provenant de https://doi.org/10.1167/5.11.3
V_lambda=np.genfromtxt('Coefficient efficace lumineuse spectrale.csv', delimiter=',', skip_header=0)

Comme un spectroradiomètre est coûteux, car l'étalonnage en flux énergétique de puissance rayonnée est fastifieux, les unités de la variable dépendante des spectres sont souvent arbitraires. Pour retrouver la grandeur radiométrique $X_e$, il faut donc d'abord normaliser le spectre par l'aire sous sa courbe en l'intégrant sur la variable dépendante mesurée, qui est généralement la longueur d'onde pour des spectromètres sensibles dans le visible.

In [30]:
spectre_LED=np.genfromtxt('LED Livowalny_Emission spectrum.txt', delimiter=None, skip_header=18, skip_footer=1)
if np.sum(spectre_LED[:,1]) !=1 :
    wavelength=spectre_LED[:,0]
    spectre_norm=spectre_LED[:,1]/np.sum(spectre_LED[:,1])
else : #si déjà normalisé
    wavelength=spectre_LED[:,0]
    spectre_norm=spectre_LED[:,1]
print(np.sum(spectre_norm),len(spectre_norm),len(wavelength),wavelength,spectre_norm) #pour vérification
#todo: faire le graphe avant et après normalisationpour confirmer que c'est la même répartition spectrale

1.0 2047 2047 [ 340.61  340.98  341.36 ... 1033.21 1033.52 1033.84] [0.00012652 0.00012875 0.00013173 ... 0.00013619 0.00013545 0.00013768]


La répartition spectrale, soit la "forme" ou densité du spectre, n'est pas affectée par ce changement d'échelle et il en va de même avec un facteur $F_e$ pour ramener le spectre normalisé $X_{arb,norm}$ à l'échelle du spectre de la grandeur radiométrique étalonnée $X_e$. On en déduit que 
\begin{equation}
    X_e(\lambda)=F_e\,X_{arb,norm}(\lambda)
\end{equation}
et il devient possible de calculer le facteur d'échelle $F_e$ à l'aide d'une grandeur photométrique en intégrant la première équation.
\begin{equation}
X_v=683F_e\int \,X_{arb,norm}(\lambda)V(\lambda)\,\mathrm{d}\lambda
\end{equation}
L'intégrale évalue le recouvrement du spectre d'émission de la source avec la réponse de l'oeil humain. Après l'interpolation de l'efficacité lumineuse spectrale $V(\lambda)$ nécessaire pour obtenir une valeur correspondant à chaque longueur d'onde mesurée du spectre de la source, cette intégrale vaut :

In [31]:
#todo: fix mon try...except block qui n'a pas l'air de faire grand chose
try :
    if np.all(np.diff(wavelength) < 0) or np.all(np.diff(V_lambda[:,0]) < 0):
        raise Exception("Les données des variables indépendantes ne sont pas ordonnés en séquence croissante, les résultats d\'interpolation seront erronés.")
except Exception as error :
    print(error)
    
V_lambda_interpole=np.interp(wavelength,V_lambda[:,0],V_lambda[:,1])
recouvrement_spectre_oeil=np.sum(spectre_norm * V_lambda_interpole)

Pour $X_v=4800$ lumens, le facteur d'échelle en watts est donc

In [32]:
X_v=4800
F_e=X_v/(683*recouvrement_spectre_oeil)
print(F_e)

16.76221906667384


In [33]:
#todo : il reste essentiellement à montrer qu'en intégrant la mise à l'échelle ci-dessus, le facteur d'échelle est bel et bien le flux énergétique de puissance rayonnée totale en intégrant l'équation de mise à l'échelle spécifiquement pour le flux énergétique. puisque le spectre normalisé intègre à 1.
# Il y a probablement une équation cute à écrire pour l'équivalence des spectres mis à l'échelle par leur facteur de conversion respectif.
# loin de la valeur des specs si c'est bien un 48W lumineux obtenu d'un 60W électrique, p-e calculé en vision scotopique?

In [34]:
# Version de Claude qui me semble manquer le scaling par le max photopique
# Calculer l'efficacité lumineuse moyenne pondérée basée sur le spectre
#    efficacite_moyenne_ponderee = np.sum(spectre_relatif * efficacites_interpolees)
# Calculer le flux radiant total (W)
#    flux_radiant_total = flux_lumineux_lumens / efficacite_moyenne_ponderee